In [72]:
import dash
import dash_table
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import dash_leaflet as dl
import requests
import json
import pandas as pd
import plotly.express as px

# Fetch data from the Flask API
def fetch_data():
    response = requests.get('http://localhost:5000/comunes')
    return pd.DataFrame(response.json())

# Extract the list of comune names
def get_comune_names():
    data = fetch_data()
    return data['nome'].tolist()

app = dash.Dash(__name__)
app.title = "Emilia Romagna Flood Risk"

def build_left_panel():
    comune_names = get_comune_names()
    return html.Div(
        id="upper-left",
        className="",
        children=[
            html.P(
                className="section-title",
                children="Choose comunes from the list below to see the information",
            ),
            html.Div(
                id="region-select-outer",
                className="control-row-2",
                children=[
                    html.Div(
                        id="checklist-container",
                        children=dcc.Checklist(
                            id="region-select-all",
                            options=[{"label": i, "value": i} for i in comune_names],
                            value=[],
                            labelStyle={'display': 'inline-block'}
                        ),
                    )
                ],
            ),
            html.H2(
                className="section-title",
                children="Indicators Overview",
            ),
            html.Div(
                id="table-upper",
                children=[
                    html.P("Demographics at Risk"),
                    dcc.Loading(
                        children=[
                            dash_table.DataTable(
                                id='table-demo-indicators',
                                columns=[
                                    {"name": "Population at Risk", "id": "pop_res011"},
                                    {"name": "Families at Risk", "id": "fam_tot"},
                                    {"name": "Area Surface at Risk", "id": "ar_kmq"}
                                ],
                                style_table={'overflowX': 'auto'},
                                style_header={'fontWeight': 'bold'},
                                style_cell={'textAlign': 'center'},
                                data=[],
                                row_deletable=True,
                                editable=False
                            )
                        ]
                    ),
                ],
            ),
            html.Div(
                id="table-down",
                children=[
                    html.P("Built Areas at Risk"),
                    dcc.Loading(
                        children=[
                            dash_table.DataTable(
                                id='table-built-indicators',
                                columns=[
                                    {"name": "Buildings at Risk", "id": "ed_tot"},
                                    {"name": "Industries at Risk", "id": "im_tot"},
                                    {"name": "Cultural Heritage at Risk", "id": "n_vir"}
                                ],
                                style_table={'overflowX': 'auto'},
                                style_header={'fontWeight': 'bold'},
                                style_cell={'textAlign': 'center'},
                                data=[],
                                row_deletable=True,
                                editable=False
                            )
                        ]
                    ),
                ],
            )
        ],
    )

def build_right_panel():
    return html.Div(
        id="upper-right",
        className="",
        children=[
             dl.Map(
                id='map',
                center=[44.4949, 11.3426],
                zoom=8,
                style={'width': '100%', 'height': '400px'},
                zoomControl=False,
                children=[
                    dl.TileLayer(url="https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png"),
                    dl.LayerGroup(id="layer"),
                    dl.LayerGroup(id="search-layer"),
                    dl.LayerGroup(id="marker-layer"),  # Add marker layer
                    dl.ZoomControl(position="topright")  # Put the ZoomControl component on topright
                ]
            ),
            html.Div(
                id="combined-plot",
                children=[
                    dcc.Loading(
                        children=[
                            dcc.Graph(id='combined-histogram',  style={'height': '350px'})
                        ]
                    ),
                ],
            ),
        ]
    )


# Set up the app layout
app.layout = html.Div(
    children=[
         html.Div(
            id="banner",
            className="banner",
            children=[
                html.H6("Emilia Romagna Flood Risk"),
            ],
        ),
        html.Div(
            id="content",
            children=[
                build_left_panel(),
                build_right_panel()
            ]
        )
    ]
)

# Define callback to update the demograhic table
@app.callback(
    Output('table-demo-indicators', 'data'),
    [Input('region-select-all', 'value')]
)
def update_demo_table(selected_comunes):
    df = fetch_data()

    df['pop_res011'] = pd.to_numeric(df['pop_res011'], errors='coerce')
    df['fam_tot'] = pd.to_numeric(df['fam_tot'], errors='coerce')
    df['ar_kmq'] = pd.to_numeric(df['ar_kmq'], errors='coerce')
    df['pop_idr_p3'] = pd.to_numeric(df['pop_idr_p3'], errors='coerce')
    df['fam_idr_p3'] = pd.to_numeric(df['fam_idr_p3'], errors='coerce')
    df['ar_id_p3'] = pd.to_numeric(df['ar_id_p3'], errors='coerce')
    df['pop_idr_p2'] = pd.to_numeric(df['pop_idr_p2'], errors='coerce')
    df['fam_idr_p2'] = pd.to_numeric(df['fam_idr_p2'], errors='coerce')
    df['ar_id_p2'] = pd.to_numeric(df['ar_id_p2'], errors='coerce')
    df['pop_idr_p1'] = pd.to_numeric(df['pop_idr_p1'], errors='coerce')
    df['fam_idr_p1'] = pd.to_numeric(df['fam_idr_p1'], errors='coerce')
    df['ar_id_p1'] = pd.to_numeric(df['ar_id_p1'], errors='coerce')
    
    # Filter data based on selected comunes
    if selected_comunes:
        filtered_df = df[df['nome'].isin(selected_comunes)]
    else:
        filtered_df = df

    # Initialize the data dictionary
    table_data = [
        {
            'pop_res011': f'Total: {filtered_df["pop_res011"].sum()}',
            'fam_tot': f'Total: {filtered_df["fam_tot"].sum()}',
            'ar_kmq': f'Total: {filtered_df["ar_kmq"].sum()}'
        },
        {
            'pop_res011': f'High Risk: {filtered_df["pop_idr_p3"].sum()}',
            'fam_tot': f'High Risk: {filtered_df["fam_idr_p3"].sum()}',
            'ar_kmq': f'High Risk: {filtered_df["ar_id_p3"].sum()}'
        },
        {
            'pop_res011': f'Medium Risk: {filtered_df["pop_idr_p2"].sum()}',
            'fam_tot': f'Medium Risk: {filtered_df["fam_idr_p2"].sum()}',
            'ar_kmq': f'Medium Risk: {filtered_df["ar_id_p2"].sum()}'
        },
        {
            'pop_res011': f'Low Risk: {filtered_df["pop_idr_p1"].sum()}',
            'fam_tot': f'Low Risk: {filtered_df["fam_idr_p1"].sum()}',
            'ar_kmq': f'Low Risk: {filtered_df["ar_id_p1"].sum()}'
        }
    ]

    return table_data

# Define callback to update the demograhic table
@app.callback(
    Output('table-built-indicators', 'data'),
    [Input('region-select-all', 'value')]
)
def update_built_table(selected_comunes):
    df = fetch_data()

    df['ed_tot'] = pd.to_numeric(df['ed_tot'], errors='coerce')
    df['im_tot'] = pd.to_numeric(df['im_tot'], errors='coerce')
    df['n_vir'] = pd.to_numeric(df['n_vir'], errors='coerce')
    df['ed_idr_p3'] = pd.to_numeric(df['ed_idr_p3'], errors='coerce')
    df['im_idr_p3'] = pd.to_numeric(df['im_idr_p3'], errors='coerce')
    df['bbcc_id_p3'] = pd.to_numeric(df['bbcc_id_p3'], errors='coerce')
    df['ed_idr_p2'] = pd.to_numeric(df['ed_idr_p2'], errors='coerce')
    df['im_idr_p2'] = pd.to_numeric(df['im_idr_p2'], errors='coerce')
    df['bbcc_id_p2'] = pd.to_numeric(df['bbcc_id_p2'], errors='coerce')
    df['ed_idr_p1'] = pd.to_numeric(df['ed_idr_p1'], errors='coerce')
    df['im_idr_p1'] = pd.to_numeric(df['im_idr_p1'], errors='coerce')
    df['bbcc_id_p1'] = pd.to_numeric(df['bbcc_id_p1'], errors='coerce')
    
    # Filter data based on selected comunes
    if selected_comunes:
        filtered_df = df[df['nome'].isin(selected_comunes)]
    else:
        filtered_df = df

    # Initialize the data dictionary
    table_data = [
        {
            'ed_tot': f'Total: {filtered_df["ed_tot"].sum()}',
            'im_tot': f'Total: {filtered_df["im_tot"].sum()}',
            'n_vir': f'Total: {filtered_df["n_vir"].sum()}'
        },
        {
            'ed_tot': f'High Risk: {filtered_df["ed_idr_p3"].sum()}',
            'im_tot': f'High Risk: {filtered_df["im_idr_p3"].sum()}',
            'n_vir': f'High Risk: {filtered_df["bbcc_id_p3"].sum()}'
        },
        {
            'ed_tot': f'Medium Risk: {filtered_df["ed_idr_p2"].sum()}',
            'im_tot': f'Medium Risk: {filtered_df["im_idr_p2"].sum()}',
            'n_vir': f'Medium Risk: {filtered_df["bbcc_id_p2"].sum()}'
        },
        {
            'ed_tot': f'Low Risk: {filtered_df["ed_idr_p1"].sum()}',
            'im_tot': f'Low Risk: {filtered_df["im_idr_p1"].sum()}',
            'n_vir': f'Low Risk: {filtered_df["bbcc_id_p1"].sum()}'
        }
    ]

    return table_data

# Define callback to update the combined histogram
@app.callback(
    Output('combined-histogram', 'figure'),
    [Input('region-select-all', 'value')]
)
def update_combined_histogram(selected_comunes):
    df = fetch_data()

    # Convert columns to numeric values
    df['pop_res011'] = pd.to_numeric(df['pop_res011'], errors='coerce')
    df['fam_tot'] = pd.to_numeric(df['fam_tot'], errors='coerce')
    df['ar_kmq'] = pd.to_numeric(df['ar_kmq'], errors='coerce')
    df['pop_idr_p3'] = pd.to_numeric(df['pop_idr_p3'], errors='coerce')
    df['fam_idr_p3'] = pd.to_numeric(df['fam_idr_p3'], errors='coerce')
    df['ar_id_p3'] = pd.to_numeric(df['ar_id_p3'], errors='coerce')
    df['pop_idr_p2'] = pd.to_numeric(df['pop_idr_p2'], errors='coerce')
    df['fam_idr_p2'] = pd.to_numeric(df['fam_idr_p2'], errors='coerce')
    df['ar_id_p2'] = pd.to_numeric(df['ar_id_p2'], errors='coerce')
    df['pop_idr_p1'] = pd.to_numeric(df['pop_idr_p1'], errors='coerce')
    df['fam_idr_p1'] = pd.to_numeric(df['fam_idr_p1'], errors='coerce')
    df['ar_id_p1'] = pd.to_numeric(df['ar_id_p1'], errors='coerce')

    df['ed_tot'] = pd.to_numeric(df['ed_tot'], errors='coerce')
    df['im_tot'] = pd.to_numeric(df['im_tot'], errors='coerce')
    df['n_vir'] = pd.to_numeric(df['n_vir'], errors='coerce')
    df['ed_idr_p3'] = pd.to_numeric(df['ed_idr_p3'], errors='coerce')
    df['im_idr_p3'] = pd.to_numeric(df['im_idr_p3'], errors='coerce')
    df['bbcc_id_p3'] = pd.to_numeric(df['bbcc_id_p3'], errors='coerce')
    df['ed_idr_p2'] = pd.to_numeric(df['ed_idr_p2'], errors='coerce')
    df['im_idr_p2'] = pd.to_numeric(df['im_idr_p2'], errors='coerce')
    df['bbcc_id_p2'] = pd.to_numeric(df['bbcc_id_p2'], errors='coerce')
    df['ed_idr_p1'] = pd.to_numeric(df['ed_idr_p1'], errors='coerce')
    df['im_idr_p1'] = pd.to_numeric(df['im_idr_p1'], errors='coerce')
    df['bbcc_id_p1'] = pd.to_numeric(df['bbcc_id_p1'], errors='coerce')
    
    # Filter data based on selected comunes
    if selected_comunes:
        filtered_df = df[df['nome'].isin(selected_comunes)]
    else:
        filtered_df = df

    # Prepare data for the plot
    data = {
        'Category': [
            'Population', 'Families', 'Area Surface', 
            'Buildings', 'Industries', 'Cultural Heritage'
        ],
        'High Risk': [
            filtered_df["pop_idr_p3"].sum(), filtered_df["fam_idr_p3"].sum(), filtered_df["ar_id_p3"].sum(), 
            filtered_df["ed_idr_p3"].sum(), filtered_df["im_idr_p3"].sum(), filtered_df["bbcc_id_p3"].sum()
        ],
        'Medium Risk': [
            filtered_df["pop_idr_p2"].sum(), filtered_df["fam_idr_p2"].sum(), filtered_df["ar_id_p2"].sum(), 
            filtered_df["ed_idr_p2"].sum(), filtered_df["im_idr_p2"].sum(), filtered_df["bbcc_id_p2"].sum()
        ],
        'Low Risk': [
            filtered_df["pop_idr_p1"].sum(), filtered_df["fam_idr_p1"].sum(), filtered_df["ar_id_p1"].sum(), 
            filtered_df["ed_idr_p1"].sum(), filtered_df["im_idr_p1"].sum(), filtered_df["bbcc_id_p1"].sum()
        ]
    }
    
    plot_df = pd.DataFrame(data)
    plot_df = plot_df.melt(id_vars='Category', var_name='Risk Level', value_name='Value')

    # Create the plot
    fig = px.bar(plot_df, x='Category', y='Value', color='Risk Level', barmode='group',
                 labels={'Value': 'Sum', 'Category': 'Indicators', 'Risk Level': 'Risk Levels'},
                 title='Combined Indicators by Risk Levels')

    return fig

# Visualize selected comune
@app.callback(
    [Output('map', 'children')],
    [Input('region-select-all', 'value')]
)
def update_map_polygons(selected_comunes):
    df = fetch_data()

    # Check if any comuni are selected
    if not selected_comunes:
        return [dl.LayerGroup(children=[])]  # Return an empty layer group if no comuni are selected

    # Filter data based on selected comuni
    filtered_df = df[df['nome'].isin(selected_comunes)]
    
    # Check if filtered data is empty
    if filtered_df.empty:
        return [dl.LayerGroup(children=[])]  # Return an empty layer group if filtered data is empty

    # Create GeoJSON data for the selected comuni
    geojson_layers = []
    for _, row in filtered_df.iterrows():
        try:
            geom = json.loads(row['geom'])  # Convert GeoJSON string to dictionary
            if geom:
                geojson_layer = dl.GeoJSON(
                    data=geom,
                    style={"color": "blue", "weight": 2, "fillOpacity": 0.1}  # Style as needed
                )
                geojson_layers.append(geojson_layer)
        except (json.JSONDecodeError, TypeError) as e:
            print(f"Error decoding GeoJSON for comune {row['nome']}: {e}")
    
    return [dl.LayerGroup(children=geojson_layers)]



if __name__ == "__main__":
    app.run_server(debug=True)
